In [2]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=os.environ['OPENAI_API_KEY_TEG'],
    temperature=0,
)


In [ ]:
search_template = """Your job is to search the web for stock market related news that would be relevant to to generate the article described by the user.

                     NOTE: When providing the news articles retrieved to the Outliner, you need to give the Outliner all of the articles you have retrieved that are relevant.
                     The format should be:

                     1. <article 1>
                     2. <article 2>
                     ...
                  """
value_template = """Your job is to edit an article written by a writer. Please provide constructive critiques so they can improve it for publication.

                    ```EDITORIAL GUIDELINES TO FOLLOW```

                      - Your feedback should be in bullet point format only.

                      - The critiques should only focus on are the use of keywords, the title of the article, and the title of the headers, also make sure they include references.

                      - DO NOT not give positive feedback.

                      - DO NOT WRITE THE ARTICLE. You should only provide feedback in bullet point format.

                      - You should NEVER accept the first draft of the article.


                     If you think the article looks good, then simply say DONE.
                    """

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define method for creating agents and binding them to the tools
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    
    if tools:
      return prompt | llm.bind_tools(tools)
    else:
      return prompt | llm
    
search_agent = create_agent(
    llm,
    tools,
    system_message=search_template,
)

value_agent = create_agent(
    llm,
    [],
    system_message=value_template,
)

